<a href="https://colab.research.google.com/github/BhaktiGajipara/LLM/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

# Define the paths to the ZIP files
zip_path1 = '/content/drive/My Drive/training_data.zip'
zip_path2 = '/content/drive/My Drive/testing_data.zip'

# Define the extraction paths
extract_path1 = '/content/training_data'
extract_path2 = '/content/testing_data'

# Create the extraction directories if they don't exist
os.makedirs(extract_path1, exist_ok=True)
os.makedirs(extract_path2, exist_ok=True)

# Unzip the first file
with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
    zip_ref.extractall(extract_path1)

# Unzip the second file
with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
    zip_ref.extractall(extract_path2)

print('Unzipping of both files complete!')

In [ ]:
import tensorflow
from tensorflow import keras
from keras.src.utils import image_dataset
from keras import Sequential
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(weights = "imagenet",
                  include_top = False,
                  input_shape = (150,150,3))

In [ ]:
conv_base.summary()

In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation = "relu"))
model.add(Dense(1,activation ="sigmoid"))

In [ ]:
model.summary()

In [ ]:
# generator
train_ds = keras.utils.image_dataset_from_directory(
    directory = "/content/training_data",
    labels = "inferred",
    label_mode = "int",
    batch_size = 32,
    image_size = (150,150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = "/content/testing_data",
    labels = "inferred",
    label_mode = "int",
    batch_size = 32,
    image_size = (150,150)
)

In [ ]:
# Normalize

def process(image,label1):
  image = tensorflow.cast(image/255. ,tensorflow.float32)
  return image,label1

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
history = model.fit(train_ds,epochs = 10, validation_data = validation_ds)